# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
csv_path = "../Weatherpy/Weather_DataFrame.csv"
weather = pd.read_csv(csv_path)
weather_df = pd.DataFrame(weather)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,khatanga,71.97,102.50,-31.13,84,74,8.39,RU,1609284659
1,rikitea,-23.12,-134.97,78.06,75,84,7.76,PF,1609284659
2,busselton,-33.65,115.33,70.00,70,19,3.00,AU,1609284659
3,airai,-8.93,125.41,64.00,98,100,0.20,TL,1609284659
4,ushuaia,-54.80,-68.30,51.80,50,75,24.16,AR,1609284493
...,...,...,...,...,...,...,...,...,...
548,libreville,0.39,9.45,80.60,88,40,6.93,GA,1609284826
549,salalah,17.02,54.09,69.80,46,0,2.24,OM,1609284826
550,atambua,-9.11,124.89,78.67,79,100,2.13,ID,1609284827
551,makrana,27.05,74.72,38.71,72,0,4.72,IN,1609284827


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key = g_key)
weather_df_na = weather_df.dropna()
humidity = weather_df_na["Humidity"].astype(float)
locations = weather_df_na[["Lat","Lng"]].astype(float)
# fig = gmaps.figure(center=(10, 0.1), zoom_level=1.8)
# heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,max_intensity=150, point_radius = 5)
# heat_layer.point_radius = 2
heat_figure_layout = {
    'width':'400px',
    'height':'300px',
    'border':'1px solid black',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout=heat_figure_layout)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', width='400px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
updated_weather_df = weather_df.loc[(weather_df["Cloudiness"]<12)]
updated_weather_df_1 = updated_weather_df.loc[(updated_weather_df["Max Temp"]>70)]
final_weather_df = updated_weather_df_1.loc[(updated_weather_df_1["Wind Speed"]<5)]
final_weather_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
304,bonthe,7.53,-12.51,78.71,82,2,4.99,SL,1609284752
319,oussouye,12.48,-16.55,75.20,94,0,4.70,SN,1609284756
326,caravelas,-17.71,-39.25,74.37,87,0,4.50,BR,1609284758
342,tepalcatepec,19.18,-102.85,82.09,34,0,2.59,MX,1609284763
354,rio grande,-32.03,-52.10,75.00,85,3,3.00,BR,1609284557
407,escarcega,18.61,-90.75,71.55,61,0,2.28,MX,1609284785
445,atiquizaya,13.98,-89.75,80.60,44,0,4.70,SV,1609284797
461,barcarena,-1.51,-48.63,77.00,88,0,4.70,BR,1609284801
491,cayenne,4.93,-52.33,77.00,100,0,4.70,GF,1609284811
507,ulundi,-28.34,31.42,72.91,82,0,1.45,ZA,1609284815


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotels_df = pd.DataFrame(final_weather_df)
hotels_df["Hotel Name"] = ""
hotels_df["Hotel lat"] = ""
hotels_df["Hotel lng"] = ""

hotels_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel lat,Hotel lng
304,bonthe,7.53,-12.51,78.71,82,2,4.99,SL,1609284752,,,
319,oussouye,12.48,-16.55,75.20,94,0,4.70,SN,1609284756,,,
326,caravelas,-17.71,-39.25,74.37,87,0,4.50,BR,1609284758,,,
342,tepalcatepec,19.18,-102.85,82.09,34,0,2.59,MX,1609284763,,,
354,rio grande,-32.03,-52.10,75.00,85,3,3.00,BR,1609284557,,,
407,escarcega,18.61,-90.75,71.55,61,0,2.28,MX,1609284785,,,
445,atiquizaya,13.98,-89.75,80.60,44,0,4.70,SV,1609284797,,,
461,barcarena,-1.51,-48.63,77.00,88,0,4.70,BR,1609284801,,,
491,cayenne,4.93,-52.33,77.00,100,0,4.70,GF,1609284811,,,
507,ulundi,-28.34,31.42,72.91,82,0,1.45,ZA,1609284815,,,


In [8]:
location = hotels_df[["Lat","Lng"]].astype(float)
params = {
#     "keyword":target_search,
    "radius":5000,
    "type":"lodging",
    "key":g_key
}


for index, row in hotels_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]= f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_names = requests.get(base_url, params=params)
    hotel_names=hotel_names.json()
    
    try:
        hotels_df.loc[index, "Hotel Name"] = hotel_names["results"][0]["name"]
        hotels_df.loc[index, "Hotel lat"] = hotel_names["results"][0]["geometry"]["location"]["lat"]
        hotels_df.loc[index, "Hotel lng"] = hotel_names["results"][0]["geometry"]["location"]["lng"]
    except:
        print(f"missing field...")
        pass
    

hotels_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel lat,Hotel lng
304,bonthe,7.53,-12.51,78.71,82,2,4.99,SL,1609284752,Bonthe Holiday Village,7.52564,-12.5006
319,oussouye,12.48,-16.55,75.20,94,0,4.70,SN,1609284756,Auberge du Routard,12.4888,-16.55
326,caravelas,-17.71,-39.25,74.37,87,0,4.50,BR,1609284758,Pousada dos Navegantes,-17.738,-39.2589
342,tepalcatepec,19.18,-102.85,82.09,34,0,2.59,MX,1609284763,Hotel Campestre,19.1742,-102.848
354,rio grande,-32.03,-52.10,75.00,85,3,3.00,BR,1609284557,Hotel Atlântico Rio Grande,-32.0312,-52.0961
407,escarcega,18.61,-90.75,71.55,61,0,2.28,MX,1609284785,Hotel Maria Isabel,18.6054,-90.7289
445,atiquizaya,13.98,-89.75,80.60,44,0,4.70,SV,1609284797,El Buho Cafe y Hostal,13.9727,-89.7563
461,barcarena,-1.51,-48.63,77.00,88,0,4.70,BR,1609284801,Dos Santos Hotel,-1.51162,-48.608
491,cayenne,4.93,-52.33,77.00,100,0,4.70,GF,1609284811,Hôtel Le Dronmi,4.9388,-52.3314
507,ulundi,-28.34,31.42,72.91,82,0,1.45,ZA,1609284815,Ekhethelo Lodge,-28.3298,31.4054


In [10]:
# # for my own practice of running through API calls

# # hotel_locations = [hotels_df["Hotel lat"],hotels_df["Hotel lng"]]
# # params = {"location": hotel_locations,"key":g_key}
# hotel_coords = hotels_df[["Hotel lat","Hotel lng"]]
# params_2 = {"address":hotel_coords,"key":g_key}

# for index, row in hotels_df.iterrows():
#     hotel_lat = row["Hotel lat"]
#     hotel_lng = row["Hotel lng"]
#     params_2["hotel_coords"]= f"{hotel_lat},{hotel_lng}"
#     base_url = "https://maps.googleapis.com/maps/api/geocode/json"
#     hotel_location = requests.get(base_url, params=params_2)
#     if hotel_location.status_code == 200: 
#         hotel_location_geo = hotel_location.json()
#     for result in hotel_location_geo["results"]:
#         for component in result["address_components"]:
#             for t in component["types"]:
#                 if t == "locality":
#                     city = component["long_name"]
#                 if  t == "country":
#                     country = component["long_name"]
#                 try:
#                     hotels_df[row_indexer, "Hotel City"] = city
#                     hotels_df[row_indexer, "Hotel Country"] = country
#                 except:
#                     print(f"missing field...")
#                     pass

    
# # print(hotel_location_geo)
# # hotel_location_geo["results"][0]["address_components"]
# #     
# hotels_df

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_content = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_content.format(**row) for index, row in hotels_df.iterrows()]
map_locations = hotels_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
gmaps.configure(api_key = g_key)
# hotel_country = hotels_df["Country"].tolist()
# hotel_city = hotels_df["City"].tolist()
# hotel_name = hotels_df["Hotel Name"].tolist()
# hotel_info = zip(hotel_country,hotel_city)
# hotel_info_complete = zip(hotel_info,hotel_name )

# ["City"]["Hotel Name"]
# 

hotel_layer = gmaps.symbol_layer(
    map_locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color = 'rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content = hotel_info
)
figure_layout = {
    'width':'400px',
    'height':'300px',
    'border':'1px solid black',
    'margin':'0 auto 0 auto'
}
#  info_box_content=[f"Country,City,Name:{hotel}" for hotel in hotel_info
# fig=gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(map_locations)
fig.add_layer(hotel_layer)
fig



Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', width='400px'))